First setup the basic selenium driver

In [149]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located, element_to_be_clickable

IDRAC_IP = "192.168.1.999"
IDRAC_USER = "username"
IDRAC_PW = "password"

service = Service(executable_path="chromium.chromedriver")
service.start()
options = Options()
options.add_argument('ignore-certificate-errors') # because sketchy idrac stuff

driver = webdriver.Chrome(service=service, options=options)

First open the web interface and wait until the last element of the login UI has loaded which is the Domain Selector option showing "This iDRAC"

In [150]:
driver.get(f"http://{IDRAC_IP}")

WebDriverWait(driver,20).until(presence_of_element_located((By.XPATH,"//option[text()='This iDRAC']")))


<selenium.webdriver.remote.webelement.WebElement (session="c0bf4db32d896d20a09bb94be40274bd", element="5c5998d3-337b-4f59-ac91-0ba3fed5c94c")>

Now find the inputs for username and password as well as the submit button, fill them out and submit the login

In [151]:


user_input_field = driver.find_element(by=By.XPATH,value="//input[@id='user']")
pw_input_field = driver.find_element(by=By.ID,value="password")
submit_button = driver.find_element(by=By.ID, value="btnOK")
user_input_field.send_keys(IDRAC_USER)
pw_input_field.send_keys(IDRAC_PW)
submit_button.click()


Here because in the idrac interface everything is in weird frames we'll have to start switching around between those, apart from that this selects the "Power" tab in the top navbar

In [152]:
navbar_frame = driver.find_element(by=By.XPATH,value="//frame[@src='snb.html']")
driver.switch_to.frame(navbar_frame)

navbar_power = driver.find_element(by=By.XPATH,value="//tbody/tr/td/a[text()='Power']")
navbar_power.click()

Now again switch frames.. (first to actually find the desired frame have to go back to normal view with the `default_content()`)

In [153]:
driver.switch_to.default_content()
power_control_frame = WebDriverWait(driver,20).until(presence_of_element_located((By.XPATH,"//frame[@src='sysSummaryData.html']")))
driver.switch_to.frame(power_control_frame)

And finally in the power menu select the radio for graceful shutdown, click the apply button and accept the alert that opens

In [154]:
graceful_shutdown_select = WebDriverWait(driver,10).until(presence_of_element_located((By.XPATH,"//input[@value='shutdown']")))
print("found button")
WebDriverWait(driver,10).until(element_to_be_clickable(graceful_shutdown_select))
print("clickable")
graceful_shutdown_select.click()
submit_power_action_button = driver.find_element(by=By.XPATH,value="//div[@class='button_clear']/a[@class='container_button']")
submit_power_action_button.click()

driver.switch_to.alert.accept()

found button
clickable


finally quit the browser

In [155]:
driver.quit()